# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.).

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
`
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'.
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [1]:
import pandas as pd
import requests

# 1) seu código aqui

sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)

sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [2]:
# Contar o número de registros
total_registros = len(sinasc)

# Contar o número de registros únicos (não duplicados)
registros_unicos = len(sinasc.drop_duplicates())

# Imprimir os resultados
print("Número total de registros:", total_registros)
print("Número de registros únicos (não duplicados):", registros_unicos)

Número total de registros: 27028
Número de registros únicos (não duplicados): 27028


In [3]:
# 2) seu código aqui

# Contar o número de valores missing por variável
valores_missing = sinasc.isna().sum()

# Exibir os resultados
print(valores_missing)

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64


In [4]:
# 3) seu código aqui

# Selecionar apenas as colunas de interesse
colunas_interesse = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
sinasc_filtrado = sinasc.loc[:, colunas_interesse]

# Contar o número de valores missing por variável nas colunas filtradas
valores_missing_filtrado = sinasc_filtrado.isna().sum()

# Exibir os resultados
print(valores_missing_filtrado)

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64


In [5]:
# 4) seu código aqui

# Remover linhas com valores ausentes na coluna Apgar5
sinasc_filtrado = sinasc_filtrado.dropna(subset=['APGAR5'])

# Contar o número de linhas após a substituição
numero_linhas = len(sinasc_filtrado)

# Contar o número de valores missing por variável nas colunas filtradas
valores_missing_filtrado = sinasc_filtrado.isna().sum()

# Exibir os resultados
print('Número de linhas: {}'.format(numero_linhas))
print('Valores missing por variável:')
print(valores_missing_filtrado)

Número de linhas: 26925
Valores missing por variável:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


In [6]:
# 5) seu código aqui

# Substituir valores ausentes nas colunas ESTCIVMAE e CONSULTAS por 9
sinasc_filtrado.loc[:, 'ESTCIVMAE'] = sinasc_filtrado['ESTCIVMAE'].fillna(9)
sinasc_filtrado.loc[:, 'CONSULTAS'] = sinasc_filtrado['CONSULTAS'].fillna(9)

# Contar o número de linhas após a substituição
numero_linhas = len(sinasc_filtrado)

# Contar o número de valores missing por variável nas colunas filtradas
valores_missing_filtrado = sinasc_filtrado.isna().sum()

# Exibir os resultados
print('Número de linhas: {}'.format(numero_linhas))
print('Valores missing por variável:')
print(valores_missing_filtrado)


Número de linhas: 26925
Valores missing por variável:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


In [7]:
# 6) Seu código aqui

# Substituir valores ausentes na coluna QTDFILVIVO por 0
sinasc_filtrado.loc[:, 'QTDFILVIVO'] = sinasc_filtrado['QTDFILVIVO'].fillna(0)

# Contar o número de linhas após a substituição
numero_linhas = len(sinasc_filtrado)

# Contar o número de valores missing por variável nas colunas filtradas
valores_missing_filtrado = sinasc_filtrado.isna().sum()

# Exibir os resultados
print('Número de linhas: {}'.format(numero_linhas))
print('Valores missing por variável:')
print(valores_missing_filtrado)

Número de linhas: 26925
Valores missing por variável:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


In [8]:
# 7) seu código aqui

# Substituir valores ausentes nas colunas categóricas LOCNASC e GRAVIDEZ pela moda
locnasc_moda = sinasc_filtrado['LOCNASC'].mode()[0]
sinasc_filtrado.loc[:, 'LOCNASC'] = sinasc_filtrado['LOCNASC'].fillna(locnasc_moda)

gravidez_moda = sinasc_filtrado['GRAVIDEZ'].mode()[0]
sinasc_filtrado.loc[:, 'GRAVIDEZ'] = sinasc_filtrado['GRAVIDEZ'].fillna(gravidez_moda)

# Substituir valores ausentes na coluna IDADEMAE pela mediana
idademae_mediana = sinasc_filtrado['IDADEMAE'].median()
sinasc_filtrado.loc[:, 'IDADEMAE'] = sinasc_filtrado['IDADEMAE'].fillna(idademae_mediana)

# Substituir valores ausentes na coluna ESCMAE por 9 (não preenchido)
sinasc_filtrado.loc[:, 'ESCMAE'] = sinasc_filtrado['ESCMAE'].fillna(9)

# Substituir valores ausentes na coluna GESTACAO pela moda
gestacao_moda = sinasc_filtrado['GESTACAO'].mode()[0]
sinasc_filtrado.loc[:, 'GESTACAO'] = sinasc_filtrado['GESTACAO'].fillna(gestacao_moda)

# Contar o número de linhas após a remoção
numero_linhas = len(sinasc_filtrado)

# Contar o número de valores missing por variável nas colunas filtradas
valores_missing_filtrado = sinasc_filtrado.isna().sum()

# Exibir os resultados
print('Número de linhas: {}'.format(numero_linhas))
print('Valores missing por variável:')
print(valores_missing_filtrado)

Número de linhas: 26925
Valores missing por variável:
LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64


In [9]:
# 8) seu código aqui

# Categorizar o APGAR5 de acordo com as faixas fornecidas
sinasc_filtrado.loc[sinasc_filtrado['APGAR5'] >= 8, 'CATEG_APGAR5'] = 'normal'
sinasc_filtrado.loc[(sinasc_filtrado['APGAR5'] >= 6) & (sinasc_filtrado['APGAR5'] <= 7), 'CATEG_APGAR5'] = 'asfixia leve'
sinasc_filtrado.loc[(sinasc_filtrado['APGAR5'] >= 4) & (sinasc_filtrado['APGAR5'] <= 5), 'CATEG_APGAR5'] = 'asfixia moderada'
sinasc_filtrado.loc[sinasc_filtrado['APGAR5'] <= 3, 'CATEG_APGAR5'] = 'asfixia severa'

# Calcular as frequências de cada categoria
frequencias_apgar = sinasc_filtrado['CATEG_APGAR5'].value_counts()

# Exibir as frequências
print('Frequências da categorização do APGAR5:')
print(frequencias_apgar)


Frequências da categorização do APGAR5:
CATEG_APGAR5
normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: count, dtype: int64


In [10]:
# 9) seu código aqui

# Renomear as variáveis para snake case
colunas_interesse = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']

# Criando um dicionário dos nomes antigos e dos novos nomes em snake case
novos_nomes = {
    'LOCNASC': 'loc_nasc',
    'IDADEMAE': 'idade_mae',
    'ESTCIVMAE': 'est_civ_mae',
    'ESCMAE': 'esc_mae',
    'QTDFILVIVO': 'qtd_fil_vivo',
    'GESTACAO': 'gestacao',
    'GRAVIDEZ': 'gravidez',
    'CONSULTAS': 'consultas',
    'APGAR5': 'apgar5'
}

# Renomear as colunas no dataframe
sinasc_filtrado.rename(columns=novos_nomes, inplace=True)

# Exibir o dataframe com as colunas renomeadas
print(sinasc_filtrado.head())


   loc_nasc  idade_mae  est_civ_mae          esc_mae  qtd_fil_vivo  \
0         1         19          5.0      8 a 11 anos           0.0   
1         1         29          2.0      8 a 11 anos           1.0   
2         1         37          9.0      8 a 11 anos           2.0   
3         1         30          5.0  12 anos ou mais           0.0   
4         1         30          2.0      8 a 11 anos           1.0   

          gestacao gravidez  consultas  apgar5 CATEG_APGAR5  
0  37 a 41 semanas    Única          4    10.0       normal  
1  37 a 41 semanas    Única          4     9.0       normal  
2  37 a 41 semanas    Única          4    10.0       normal  
3  37 a 41 semanas    Única          3    10.0       normal  
4  37 a 41 semanas    Única          4    10.0       normal  
